In [ ]:
import cherrypy
import pandas as pd
import numpy as np
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load datasets
severity_df = pd.read_csv(r"C:\Users\saeem\Downloads\Symptom-severity.csv")
dataset_df = pd.read_csv(r"C:\Users\saeem\Downloads\dataset.csv")
description_df = pd.read_csv(r"C:\Users\saeem\Downloads\symptom_Description.csv")
precaution_df = pd.read_csv(r"C:\Users\saeem\Downloads\symptom_precaution.csv")

# Preprocessing
symptoms = severity_df['Symptom'].unique()
symptom_index = {symptom: index for index, symptom in enumerate(symptoms)}
disease_encoder = LabelEncoder()
dataset_df['Disease'] = disease_encoder.fit_transform(dataset_df['Disease'])

# Preparing training data
X = np.zeros((dataset_df.shape[0], len(symptoms)))
for i, row in dataset_df.iterrows():
    for symptom in row[1:].dropna():
        if symptom in symptom_index:
            X[i, symptom_index[symptom]] = 1

y = dataset_df['Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

class DiseasePredictionApp:
    @cherrypy.expose
    def index(self):
        """ Serves the enhanced HTML frontend """
        return """
        <html>
        <head>
            <title>Disease Prediction</title>
            <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600&display=swap" rel="stylesheet">
            <style>
                body {
                    font-family: 'Poppins', sans-serif;
                    background: linear-gradient(135deg, #6a11cb, #2575fc);
                    color: #fff;
                    text-align: center;
                    display: flex;
                    flex-direction: column;
                    align-items: center;
                    justify-content: center;
                    height: 100vh;
                    margin: 0;
                }
                .container {
                    background: rgba(255, 255, 255, 0.2);
                    padding: 30px;
                    border-radius: 15px;
                    box-shadow: 0px 0px 20px rgba(0, 0, 0, 0.2);
                    width: 80%;
                    max-width: 500px;
                }
                h1 {
                    font-weight: 600;
                }
                input {
                    width: 80%;
                    padding: 10px;
                    border-radius: 5px;
                    border: none;
                    margin: 10px 0;
                    font-size: 16px;
                    text-align: center;
                }
                button {
                    background: #ff4081;
                    color: white;
                    border: none;
                    padding: 10px 20px;
                    font-size: 18px;
                    border-radius: 5px;
                    cursor: pointer;
                    transition: 0.3s;
                }
                button:hover {
                    background: #e6006e;
                }
                .result {
                    margin-top: 20px;
                    padding: 15px;
                    border-radius: 10px;
                    background: rgba(255, 255, 255, 0.3);
                }
            </style>
            <script>
                async function predictDisease() {
                    let symptoms = document.getElementById("symptoms").value;
                    let response = await fetch("/predict", {
                        method: "POST",
                        headers: { "Content-Type": "application/json" },
                        body: JSON.stringify({ symptoms: symptoms.split(",").map(s => s.trim()) })
                    });
                    let data = await response.json();
                    document.getElementById("result").innerHTML = 
                        "<div class='result'>" +
                        "<h3>Predicted Disease: " + data.Disease + "</h3>" +
                        "<p><b>Description:</b> " + data.Description + "</p>" +
                        "<p><b>Precautions:</b><ul>" + data.Precautions.map(p => "<li>" + p + "</li>").join("") + "</ul></p>" +
                        "</div>";
                }
            </script>
        </head>
        <body>
            <div class="container">
                <h1>Disease Prediction</h1>
                <input type="text" id="symptoms" placeholder="Enter symptoms (comma-separated)">
                <button onclick="predictDisease()">Predict</button>
                <div id="result"></div>
            </div>
        </body>
        </html>
        """

    @cherrypy.expose
    @cherrypy.tools.json_out()
    @cherrypy.tools.json_in()

    def predict(self):
        """ Predicts disease based on symptoms """
        input_data = cherrypy.request.json
        symptoms_input = input_data.get("symptoms", [])
        
        input_vector = np.zeros(len(symptoms))
        for symptom in symptoms_input:
            if symptom in symptom_index:
                input_vector[symptom_index[symptom]] = 1

        predicted_disease = disease_encoder.inverse_transform([model.predict([input_vector])[0]])[0]
        description = description_df[description_df['Disease'] == predicted_disease]['Description'].values[0]
        precautions = precaution_df[precaution_df['Disease'] == predicted_disease].iloc[:, 1:].values.flatten().tolist()

        return {
            "Disease": predicted_disease,
            "Description": description,
            "Precautions": precautions
        }

if __name__ == '__main__':
    cherrypy.config.update({'server.socket_host': '0.0.0.0', 'server.socket_port': 9090})
    cherrypy.quickstart(DiseasePredictionApp())
